In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [5]:
pinecone_ = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
index_name = ''
namespace = ''
pinecone_.create_index(
    name=index_name,
    dimension=1536,     # This is openAI embeddings number
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

In [ ]:
import json
data = json.load(open('reviews_example.json'))
data['reviews']

In [8]:
process_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model='text-embedding-3-small',
    )
    embedding = response.data[0].embedding
    process_data.append({
        'values': embedding,
        'id': review['professor'],
        'metadata': {
            'review': review['review'],
            'subject': review['subject'],
            'stars': review['stars']
        }
    })

In [10]:
index = pinecone_.Index('index_name')
index.upsert(
    vectors=process_data,
    namespace=namespace
)

{'upserted_count': 20}

In [11]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}